In [1]:
import pandas as pd
import numpy as np

# Marginal structural models, frequentist approach
## IPTW based MSM
This is a python conversion of Andrew Heiss' blog post on IPTW based MSM's here: https://www.andrewheiss.com/blog/2020/12/03/ipw-tscs-msm/
Andrew's generated data will be used. The dataset simulates the effects of the 6-hour working day policy (binary treatment) on happiness in several (fake) countries. The underlying DAG is this: https://www.andrewheiss.com/blog/2020/12/03/ipw-tscs-msm/index_files/figure-html/dag-complex-1.png

Here, our goal is to get an estimate for the instantaneous (same year) effect on happiness of having instituted a 6-hour work day vs not having insituted this policy.

### Load data
After loading the data, a preprocessing step to filter out countries that never adopt the new policies. This is because of math issues when treatment remains unchanged. This could be helped with zero-inflated modeling of the IPW instead of plain logistic regression.

In [2]:
data = pd.read_csv('https://www.andrewheiss.com/blog/2020/12/03/ipw-tscs-msm/happiness_data.csv')
policy_data = data[data["country"].isin(data[data["policy"] == 1].country)].reset_index(drop=True).copy()

## Binary treatment MSM/IPTW
### How and why IPTW works
Nicely explained here: https://www.youtube.com/watch?v=PfLYPt9ur4g

If $Z$ is a set of variables that completely explain the confounding, then we have conditional exchangeability given $Z$. I.e. within levels of $Z$ the observation of treatment groups $A$ and their respective outcomes can be used to easily calculate the potential outcomes. Imagine a situation with only binary confounds, treatments and outcomes. To calculate the treatment effect we need to calculate the outcomes if all individuals recieved treatment or not, within groups.

So, for example, in group $Z=0$ we need to calculate the number of successes/failures if all 40 did get the treatment. Since 0.7 of the treated see success, we would end up with another 7 successes and three failures. We get this exact result if we just multiply the number of successes/failures in the treatment group by multiplying the inverse probability of receiving treatment within this strata of $Z$! The same is applied to group $Z=1$.

<img src='iptw1.png'>

The same logic applies when calculating the outcomes had all individuals not been treated:

<img src='iptw2.png'>

If we now add up the weighted populations of treated and untreated, we end up with a "pseudopopulation". One can see that it is larger (n=200) and that the probabilty of treatment is equal between groups. (Note: there could of course remain differences in treatment effect across groups, but this is true for randomized experiments as well!)

<img src='iptw3.png'>





### Calculating IPTW for binary treatment in time series
$\text{unstabilized binary IPW}_{it} = \prod_{t=1}^{t} \frac{1}{P(X_{it} | \bar{X}_{i,t-1}, Y_{i,t-1}, C_{it}, V_{i})}$, where

 - ${i}$ is the individual country
 - ${t}$ it the timestep, ${X_{it}}$ is an observed treatment assignment for ${i}$ at ${t}$
 - $\bar{X}_{i,t-1}$ is the observed treatment assignments for ${i}$ at ${t-1}$
 - ${Y}_{i,t-1}$ is the observed outcome ${i}$ at ${t-1}$, $C_{it}$ are the time varying confounders for ${i}$ at ${t}$
 - $V_{i}$ are the time invariant (constant) confounders for ${i}$


<b>Read as:</b> "inverse probability of treatment given all previous treatment assignments, the outcome of interest at the previous timestep, the current time varying confounders and the constant confounders"

However, these weights need to be stabilized. This is done by a modifying the numerator:

$\text{stabilized binary IPW}_{it} = \prod_{t=1}^{t} \frac{P(X_{it} | \bar{X}_{i,t-1}, V_{i})}{P(X_{it})| \bar{X}_{i,t-1}, Y_{i,t-1}, C_{it}, V_{i})}$

### Recipe for IPTW MSM
1. Define the structure of the model (a DAG) 
2. Define the propensity for treatment function $e(W)$ and fit it
3. Calculate the instantaneous IPTW for each datapoint
4. Calculate the cumulative IPTW for all datapoints through all timesteps $t$
5. Reweight the population and fit the estimator to get the estimate

In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

#create new dataframe
policy_data_weighted = policy_data.copy()

#raw propensity scores
model_num = smf.glm("policy ~ lag_policy + country", data=policy_data, family=sm.families.Binomial(sm.families.links.logit()))
model_den = smf.glm("policy ~ log_gdp_cap + democracy + corruption + lag_happiness_policy + lag_policy + country", data=policy_data, family=sm.families.Binomial(sm.families.links.logit()))
policy_data_weighted["propensity_num"] = model_num.fit().fittedvalues
policy_data_weighted["propensity_den"] = model_den.fit().fittedvalues

#calculate instantaneous ipw
policy_data_weighted["propensity_num_outcome"] = np.where(policy_data_weighted["policy"]==1, policy_data_weighted["propensity_num"], 1-policy_data_weighted["propensity_num"])
policy_data_weighted["propensity_den_outcome"] = np.where(policy_data_weighted["policy"]==1, policy_data_weighted["propensity_den"], 1-policy_data_weighted["propensity_den"])
policy_data_weighted["instant_iptw"] = policy_data_weighted["propensity_num_outcome"] / policy_data_weighted["propensity_den_outcome"]

#calculate actual ipw (cumsum)
policy_data_weighted["iptw"] = policy_data_weighted.groupby("country").instant_iptw.cumprod()

### Estimation of ATE
With the IPWT calculated, we can estimate the ATE. Here I'll use a simple GLM, whereas Heiss uses a mixed effects model. The latter simply does not work with statsmodels as it will not accept weights. However, the effect of the policy on happiness is very close to the true value 7.6!


In [4]:
policy_ate_model = smf.glm("happiness_policy ~ policy + lag_policy",
                           data = policy_data_weighted,
                           freq_weights=policy_data_weighted["iptw"])

In [5]:
policy_ate_model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:       happiness_policy   No. Observations:                 1380
Model:                            GLM   Df Residuals:                  2014.91
Model Family:                Gaussian   Df Model:                            2
Link Function:               identity   Scale:                          80.223
Method:                          IRLS   Log-Likelihood:                -7285.9
Date:                Wed, 09 Nov 2022   Deviance:                   1.6164e+05
Time:                        17:44:20   Pearson chi2:                 1.62e+05
No. Iterations:                     3   Pseudo R-squ. (CS):             0.2054
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     48.5687      0.471    103.069      0.000      47.645      49.492
policy         7.8076      0.771     10.128      0.000       6.297       9.319
lag_policy     1.5761      0.654      2.410      0.016       0.294       2.858
==============================================================================
"""

### Final thoughts
- Python is no good for this type of problems. R has packages such as `lmer` and `ipw` will make your life easier.